In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []
    
def make_pairs(words):
    pairs = []
    for i in range(len(words)-1):
        pairs.append("%s_%s" % (words[i].lower(), words[i+1].lower()))
    return pairs

wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)
pairs = wiki.map(make_pairs)
num_pairs = pairs.flatMap(lambda pair: pair).filter(lambda pair: pair.startswith('narodnaya_')).map(lambda pair: (pair, 1))
sum_keys = num_pairs.reduceByKey(lambda x, y: x + y)
result = sorted(sum_keys.map(lambda x: "%s\t%d" % x).collect())

for line in result:
    print line
